<a href="https://colab.research.google.com/github/profteachkids/CHE3022_Fall2022/blob/main/NetworkFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.optimize import root

In [2]:
mu = 1e-3  #Pa s
rho = 1e3   # kg/m3
g=9.81      #m/s2

In [3]:
def churchill(Re, eD):
    t1 = (-2.457*np.log((7./Re)**0.9 + 0.27*eD))**16
    t2 = (37530/Re)**16
    return 2* ((8/Re)**12 + 1/(t1+t2)**1.5)**(1/12.)

In [21]:
def headDrop(Q, L, D, eD, K=0.):
   v=4*np.abs(Q)/(np.pi*(D**2))
   Re = rho * v * D / mu
   f = churchill(Re, eD)
   return np.copysign(2*f*L*(v**2)/(g*D) + K/2/g*(v**2),Q)

In [22]:
#hD is the decrease in head from inlet to outlet
#Positive hD indicates positive from inlet to outlet
def flow(hD, L, D, eD, K=0.):

    # guess f = 0.01
    Qguess = np.copysign(np.pi*(D**2)/4 * np.sqrt(np.abs(hD)*g/(K/2 + 0.02*L/D)),hD)
    def eq(x):
        Q=x[0]
        return [hD - headDrop(Q, L, D, eD, K)]

    return root(eq, [Qguess]).x[0]

In [39]:


pipes=np.array([[1, 3, 35., 0.05, 1e-6, 3.],
                [2, 4, 20., 0.05, 1e-6, 2.],
                [3, 4, 5., 0.05, 1e-6, 1.],
                [3, 5, 10., 0.05, 1e-6, 4.],
                [3, 6, 1., 0.05, 1e-6, 1.],
                [4, 5, 3., 0.05, 1e-6, 2.],
                [5, 6, 1., 0.05, 1e-6, 1.]])

#indexing starts from 0
pipe_in = pipes[:,0].astype(np.int32)-1
pipe_out = pipes[:,1].astype(np.int32)-1
lengths = pipes[:,2]
diameters = pipes[:,3]
eDs = pipes[:,4]
Kvals=pipes[:,5]

In [35]:
pipe_in

array([1, 2, 3, 3, 3, 4, 5], dtype=int32)

In [31]:
z=np.array([30., 10., 0., 5., 0., 0.])
external_nodes=[1, 2, 6]


In [37]:
z[pipe_in]

array([30., 10.,  0.,  0.,  0.,  5.,  0.])